In [ ]:
!pip install datasets transformers accelerate tokenizers

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer,Trainer, TrainingArguments

In [ ]:
from datasets import load_dataset

In [ ]:
datasets=load_dataset("text",data_files="/content/dr_patient.txt")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 78
    })
})

In [ ]:
tokenizer=GPT2Tokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
model=GPT2LMHeadModel.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
  tokenizer_inputs=tokenizer(examples["text"],truncation=True,padding="max_length",max_length=128)
  tokenizer_inputs["labels"]=tokenizer_inputs["input_ids"].copy()
  return tokenizer_inputs

In [ ]:
tokenized_dataset=datasets.map(tokenize_function,batched=True,num_proc=4,remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/78 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 78
    })
})

In [ ]:
from peft import get_peft_model,LoraConfig,TaskType

In [ ]:
lora_config=LoraConfig(
    r=8,
    lora_alpha=64,
    target_modules=["c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
peft_model=get_peft_model(model,lora_config)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
peft_model.print_trainable_parameters()

trainable params: 589,824 || all params: 125,029,632 || trainable%: 0.4717


In [ ]:
training_args=TrainingArguments(
    output_dir="./Lora_dr_patient_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2
)

In [ ]:
trainer=Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yadav-abhinay-01012001 (yadav-abhinay-01012001-iit-mandi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=100, training_loss=8.131400146484374, metrics={'train_runtime': 531.3146, 'train_samples_per_second': 0.734, 'train_steps_per_second': 0.188, 'total_flos': 25652637204480.0, 'train_loss': 8.131400146484374, 'epoch': 5.0})

In [ ]:
model.save_pretrained("./Lora_dr_patient_finetuned")




In [ ]:
input_prompt="Doctor : I have headache"

from transformers import pipeline

generator=pipeline("text-generation",model="./Lora_dr_patient_finetuned",tokenizer=tokenizer)


result=generator(input_prompt,max_length=100,do_sample=True)
print(result[0]["generated_text"])

Some weights of the model checkpoint at ./Lora_dr_patient_finetuned were not used when initializing GPT2LMHeadModel: ['transformer.h.0.attn.c_attn.base_layer.bias', 'transformer.h.0.attn.c_attn.base_layer.weight', 'transformer.h.0.attn.c_attn.lora_A.default.weight', 'transformer.h.0.attn.c_attn.lora_B.default.weight', 'transformer.h.1.attn.c_attn.base_layer.bias', 'transformer.h.1.attn.c_attn.base_layer.weight', 'transformer.h.1.attn.c_attn.lora_A.default.weight', 'transformer.h.1.attn.c_attn.lora_B.default.weight', 'transformer.h.10.attn.c_attn.base_layer.bias', 'transformer.h.10.attn.c_attn.base_layer.weight', 'transformer.h.10.attn.c_attn.lora_A.default.weight', 'transformer.h.10.attn.c_attn.lora_B.default.weight', 'transformer.h.11.attn.c_attn.base_layer.bias', 'transformer.h.11.attn.c_attn.base_layer.weight', 'transformer.h.11.attn.c_attn.lora_A.default.weight', 'transformer.h.11.attn.c_attn.lora_B.default.weight', 'transformer.h.2.attn.c_attn.base_layer.bias', 'transformer.h.2.at

Doctor : I have headache-1, of the Bible story is to the fact, are not to not just get in- It's there is that this was a complete the public of the land on the other countries of the land and see the ground- The H.
" the ground- a higher education, "It is very few, my heart- not only one's you can change of the other than you think their position of a man- for security."
In the law enforcement and elimination or about the sunburns not to the planet is that there is that its own property of the Air Force Field is really, "The second-
2. You have a "The "This is the other. I've been in very small for nearly all-
6, "The following the publicized (f
' s/t.is the ground-1: You're not to the land, "I wantonly does not be his, say in the skies.
In the sky-H-0, "Including the heavens; they are to the sky-5-5-1-Seems to these guys on a public of a carrying is the surface, "B.
This is just about 5. The Bible and from the skies of a "
